In [1]:
from IPython.display import clear_output
from matplotlib import pyplot as plt
import numpy as np
import collections
%matplotlib inline

def live_plot(data_dict, figsize=(7,5), title=''):
    clear_output(wait=True)
    plt.figure(figsize=figsize)
    for label,data in data_dict.items():
        plt.plot(data, label=label)
    plt.title(title)
    plt.grid(True)
    plt.xlabel('epoch')
    plt.legend(loc='center left') # the plot evolves to the right
    plt.show();

In [2]:
import plotly.graph_objects as go
def plot(x,y,z,data = None, constr = None):
    fig = go.Figure()
    fig.add_trace(go.Surface(x = x, y = y,z=z))
    if data is not None: 
        fig.add_trace(go.Scatter3d(x=data[:,0], y=data[:,1], z=data[:,2],
                                   mode='markers'))
    if constr is not None: 
        fig.add_trace(go.Scatter3d(x=constr[:,0], y=constr[:,1], z=constr[:,2],
                                   mode='markers')) 

    fig.update_layout(title='Surface Plot', autosize=True,
                  width=800, height=800, font=dict(
                  family="Courier New, monospace",
                  size=18),
                  margin=dict(l=65, r=50, b=65, t=90))

    fig.show()

In [3]:
def make_plot(data = None):
    x1,x2 = np.linspace(-500,500,100),np.linspace(-500,500,100)
    x_pred = np.transpose([np.tile(x1, len(x2)), np.repeat(x2, len(x1))])
    r = np.sqrt(160000.)
    c1,c2 = r*np.cos(np.linspace(0,2.*np.pi,100)),r*np.sin(np.linspace(0,2.*np.pi,100))
    c3 = np.zeros((len(c2)))
    x1,x2 = np.meshgrid(x1,x2)
    z = np.zeros((10000))
    c = np.zeros((10000))
    for i in range(10000): 
        z[i] = schwefel(x_pred[i], 1, 1)
        c[i] = g(x_pred[i],1,1)
    for i in range(len(c1)):
        c3[i]= schwefel(np.array([c1[i],c2[i]]), 1, 1)
    plot(x1,x2,z.reshape(100,100).T, data = data, constr = np.column_stack([c1,c2,c3]))
    plot(x1,x2,c.reshape(100,100).T)

In [4]:
import numpy as np
from hgdl.hgdl import HGDL as hgdl
from hgdl.support_functions import *
from hgdl.constraints import NonLinearConstraint
import time
import dask.distributed as distributed
import tracemalloc

def g(x, arr, barr):
    return (np.linalg.norm(x)**2/10.0) - 16000.0

def g_grad(x, arr, barr):
    return (2./10.) * x

def g_hess(x, arr, barr):
    return np.identity(len(x)) * (2./10.)

con = NonLinearConstraint(g,g_grad,g_hess)

from scipy.optimize import minimize

arr  = 5
brr  = 6
bounds = np.array([[-500,500],[-500,500]])
a = hgdl(schwefel, schwefel_gradient, bounds,
            #hess = schwefel_hessian,
            hess = schwefel_hessian,
            #global_optimizer = "random",
            global_optimizer = "genetic",
            #global_optimizer = "gauss",
            local_optimizer = "L-BFGS-B",
            number_of_optima = 30000,
            args = (arr,brr), radius = None, num_epochs = 100, constraints = ())

#for constrained optimization, we recommend letting the algorithms choose the starting positions
#x0 = np.random.uniform(low = np.ones((2)) * -200., high = np.ones((2)) * 200.,size = (20,2))
    
a.optimize()



#print(a.optima_list)
print("main thread submitted HGDL and will now sleep for 5 seconds")
time.sleep(5)
print("main thread asks for 10 best solutions:")
print(a.get_latest())
#a.cancel_tasks()
print("main sleeps for another 2 seconds")
time.sleep(2)
print("main thread kills optimization")
res = a.kill_client()
print("hgdl was killed but I am waiting 2s")
print("")
print("")
print("")
print("")

#print(fe)





main thread submitted HGDL and will now sleep for 5 seconds
JHGFHGFGHJFH:  [[ 1.36986152e-06  1.11640186e-07]
 [-1.49688003e-06  3.50608604e-07]
 [ 1.07868091e+01  2.35507557e-06]]
JHGFHGFGHJFH:  [[ 1.89279140e-07  1.18238678e-07]
 [-2.17573229e-08  4.48827788e-08]
 [ 8.44176499e+00 -1.34805592e+00]]


distributed.worker - WARNING - Compute Failed
Function:  hgdl
args:      ({'transfer data': <distributed.variable.Variable object at 0x7f7971ddb850>, 'break condition': <distributed.variable.Variable object at 0x7f7971ddbaf0>, 'optima': <hgdl.optima.optima object at 0x7f7971ddbcd0>, 'metadata': <hgdl.meta_data.meta_data object at 0x7f7971ddbd60>})
kwargs:    {}
Exception: 'AttributeError("\'dict\' object has no attribute \'list\'")'



main thread asks for 10 best solutions:
{'x': array([[ 420.96874043,  203.81425402],
       [-302.5249302 , -302.52493517],
       [-500.        , -302.52492631],
       [-124.82935569, -124.82935596],
       [  65.54786501,  -25.87741719]]), 'f(x)': array([217.13969485, 236.87669468, 356.83208881, 592.21345297,
       750.24785783]), 'classifier': ['optimum', 'optimum', 'degenerate', 'optimum', 'optimum'], 'Hessian eigvals': array([[nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan]]), 'df/dx': array([ 1.11640186e-07,  1.36986152e-06, -1.49688003e-06,  1.89279140e-07,
        1.18238678e-07]), '|df/dx|': array([1.53739266e-06, 1.37440319e-06, 1.07868091e+01, 2.23174770e-07,
       4.98783013e-08]), 'success': True}


AttributeError: 'dict' object has no attribute 'list'

In [ ]:
print(np.column_stack([res["x"],res["k"]]))

In [ ]:
print(res["x"])
fe = np.ones(len(res["x"]))
for i in range(len(res["x"])): fe[i] = schwefel(res["x"][i,0:2],1,1)
make_plot(data = np.column_stack([res["x"][:,0:2],fe]))